In [48]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import torchvision.models as models
import pandas as pd




class EmailDataset(Dataset): 
    #annotations_file - Path to .csv of email features (https://github.com/diegoocampoh/MachineLearningPhishing)
    def __init__(self, annotations_file, train=True, transform=None):
        self.data_frame = pd.read_csv(annotations_file)
        self.train = train
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self,index):
        
        label = self.data_frame.iloc[index, 10]
        row = self.data_frame.iloc[index, 0:10]
        atInUrl =  1 if row['@ in URLs'] else 0
        Attachments = row['Attachments']
        Css = row['Css']
        Ext = row['External Resources']
        htmlcont = 1 if row['HTML content'] else 0
        htmlform = 1 if row['Html Form'] else 0
        htmliframe = 1 if row['Html iFrame'] else 0
        ip = 1 if row['IPs in URLs'] else 0
        js = row['Javascript']
        urls = row['URLs']
        features = [atInUrl,Attachments,Css,Ext,htmlcont,htmlform,htmliframe,ip,js,urls]
        features_tensor = torch.tensor(features, dtype=torch.float32)
        if self.transform:
            features_tensor = self.transform(features_tensor)
        return features_tensor, label

training_data = EmailDataset(
    annotations_file="Enron-Phishing Dataset/features-enron-and-phishing.csv",
    train=True,
)

test_data = EmailDataset(
    annotations_file="Enron-Phishing Dataset/features-enron-and-phishing.csv",
    train=False,
)

train_dataloader = DataLoader(training_data, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=32, shuffle=True)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        # 512 Nodes on 1 hidden Layer
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(10, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 2),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()
learning_rate = 1e-4 #10^-3 Learning Rate
batch_size = 32
loss_fn = nn.CrossEntropyLoss() #Cross Entropy Loss Function
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
epochs = 10 #5 Runs of the dataset

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 45 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")


# model = models.vgg16(weights='IMAGENET1K_V1')
# torch.save(model.state_dict(), 'model_weights.pth')


Epoch 1
-------------------------------
loss: 0.891562  [   32/ 4512]
loss: 0.669818  [ 1472/ 4512]
loss: 0.611250  [ 2912/ 4512]
loss: 0.581180  [ 4352/ 4512]
Test Error: 
 Accuracy: 88.3%, Avg loss: 0.624691 

Epoch 2
-------------------------------
loss: 0.613691  [   32/ 4512]
loss: 0.614267  [ 1472/ 4512]
loss: 0.550830  [ 2912/ 4512]
loss: 0.640047  [ 4352/ 4512]
Test Error: 
 Accuracy: 88.5%, Avg loss: 0.612635 

Epoch 3
-------------------------------
loss: 0.530547  [   32/ 4512]
loss: 0.633306  [ 1472/ 4512]
loss: 0.583657  [ 2912/ 4512]
loss: 0.593926  [ 4352/ 4512]
Test Error: 
 Accuracy: 88.5%, Avg loss: 0.606638 

Epoch 4
-------------------------------
loss: 0.865785  [   32/ 4512]
loss: 0.542463  [ 1472/ 4512]
loss: 0.585269  [ 2912/ 4512]
loss: 0.516855  [ 4352/ 4512]
Test Error: 
 Accuracy: 88.5%, Avg loss: 0.602063 

Epoch 5
-------------------------------
loss: 0.783441  [   32/ 4512]
loss: 0.771659  [ 1472/ 4512]
loss: 0.515824  [ 2912/ 4512]
loss: 0.604210  [ 4352